In [18]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import pyodbc

In [19]:
local_ = r"C:\Users\zengsh\OneDrive - Westfund Ltd\Documents\Westfund_"
output_folder = r"\\prdeqs01\QlikData\earned_contributions_rolling"

In [20]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au;'
    'DATABASE=BRONZE;'
    'Trusted_Connection=yes;'
)

In [21]:
def run_earned_contributions(period_start, period_end, conn,
                              curr_ri='2025-04-01', prev_ri='2024-04-01'):
    """
    Run earned contributions query for each month where current_month goes from
    period_start to period_end (inclusive, stepping by 1 month).

    Parameters
    ----------
    period_start : str   e.g. "2025-08-01"  (first day of the current_month loop)
    period_end   : str   e.g. "2025-10-01"
    conn         : pyodbc connection
    curr_ri      : str   current rate-increase date
    prev_ri      : str   previous rate-increase date

    Returns
    -------
    pd.DataFrame  combined across all months
    """
    start = datetime.strptime(period_start, '%Y-%m-%d')
    end   = datetime.strptime(period_end,   '%Y-%m-%d')

    dfs = []
    current_month = start

    while current_month <= end:
        previous_month   = current_month - relativedelta(months=1)
        report_month     = previous_month
        report_month_end = report_month + relativedelta(months=1) - relativedelta(days=1)

        cm  = current_month.strftime('%Y-%m-%d')
        pm  = previous_month.strftime('%Y-%m-%d')
        rm  = report_month.strftime('%Y-%m-%d')
        rme = report_month_end.strftime('%Y-%m-%d')

        query = f"""
WITH Advances_Arrears AS (
    SELECT
        a.membership_id,
        CAST('{rm}' AS date) AS Month_Year,
        (ISNULL(b.member_arrears, 0) - ISNULL(c.member_arrears, 0)) AS Net_Arrears,
        (ISNULL(b.member_advance, 0) - ISNULL(c.member_advance, 0)) AS Net_Advance
    FROM (
        SELECT DISTINCT membership_id
        FROM dbo.group_key_full_by_branch
        WHERE rundate BETWEEN '{pm}' AND '{cm}'
    ) a
    LEFT OUTER JOIN dbo.group_key_full_by_branch b
        ON a.membership_id = b.membership_id AND b.rundate = '{cm}'
    LEFT OUTER JOIN dbo.group_key_full_by_branch c
        ON a.membership_id = c.membership_id AND c.rundate = '{pm}'
),

Receipts_raw AS (
    SELECT
        r.membership_id,
        ISNULL(r.con_discount_pct, 0) / 100 AS con_discount,
        CAST((r.receipt_amount + r.rebate_amount) AS decimal(18,2)) AS received_amount,
        CAST((ISNULL(r.receipt_amount_bonus, 0) + ISNULL(r.rebate_amount_bonus, 0)) AS decimal(18,2)) AS bonus_amount
    FROM dbo.receipt AS r
    INNER JOIN dbo.receipt_status rs
        ON r.receipt_id = rs.receipt_id AND r.membership_id = rs.membership_id
    WHERE r.create_datetime >= '{rm}'
      AND r.create_datetime < DATEADD(DAY, 1, '{rme}')
      AND rs.receipt_status_type IN ('P', 'U', 'A')
),

Monthly_receipts AS (
    SELECT
        membership_id,
        MAX(con_discount)    AS con_discount,
        SUM(received_amount) AS received_amount,
        SUM(bonus_amount)    AS bonus_amount
    FROM Receipts_raw
    GROUP BY membership_id
),

All_contributions AS (
    SELECT
        COALESCE(mr.membership_id, aa.membership_id) AS membership_id,
        CAST('{rm}' AS date)            AS Month_Year,
        ISNULL(mr.received_amount, 0)   AS received_amount,
        ISNULL(mr.bonus_amount, 0)      AS bonus_amount,
        ISNULL(aa.Net_Advance, 0)       AS Net_Advance,
        ISNULL(aa.Net_Arrears, 0)       AS Net_Arrears,
        ISNULL(mr.con_discount, 0)      AS con_discount
    FROM Monthly_receipts mr
    FULL OUTER JOIN Advances_Arrears aa
        ON mr.membership_id = aa.membership_id
),

Member_details_Ranked AS (
    SELECT
        cp.membership_id, cp.product_id, p.product_type, c.cover_type, c.cover_state,
        ROW_NUMBER() OVER (PARTITION BY cp.membership_id ORDER BY cp.cover_version DESC) AS vn
    FROM dbo.cover_product cp
    LEFT OUTER JOIN dbo.product p ON cp.product_id = p.product_id
    LEFT OUTER JOIN dbo.cover c   ON c.membership_id = cp.membership_id AND c.cover_version = cp.cover_version
    WHERE c.cover_from_date <= '{cm}'
),

Member_details AS (
    SELECT * FROM Member_details_Ranked WHERE vn = 1
),

DRL AS (
    SELECT
        m.membership_id,
        (m.member1_loading / 100)               AS mem1_loading,
        (m.member2_loading / 100)               AS mem2_loading,
        ISNULL(m.overall_agediscount, 0) / 100  AS overall_agediscount,
        ISNULL(gr.grp_discount_amount, 0) / 100 AS corporate_discount
    FROM dbo.memship m
    LEFT OUTER JOIN dbo.memship_app_agent ma ON m.membership_id = ma.membership_id
    LEFT OUTER JOIN dbo.grouping gr           ON ma.agent_group_id = gr.group_id
    WHERE m.memship_status NOT IN ('L', 'P')
),

Combined_data AS (
    SELECT
        ac.*,
        (received_amount + bonus_amount + Net_Arrears - Net_Advance) AS earned_contributions,
        ISNULL(MAX(CASE WHEN md.product_type = 'H' THEN md.product_id END), 0) AS H_prod,
        ISNULL(MAX(CASE WHEN md.product_type = 'A' THEN md.product_id END), 0) AS A_prod,
        ISNULL(MAX(CASE WHEN md.product_type = 'B' THEN md.product_id END), 0) AS B_prod,
        md.cover_type,
        md.cover_state,
        CASE WHEN md.cover_type IN ('A','B','P','S')
             THEN d.mem1_loading
             ELSE (d.mem1_loading + d.mem2_loading) / 2
        END AS overall_lhc,
        d.overall_agediscount,
        d.corporate_discount
    FROM All_contributions ac
    LEFT OUTER JOIN Member_details md ON ac.membership_id = md.membership_id
    LEFT OUTER JOIN DRL d             ON ac.membership_id = d.membership_id
    GROUP BY
        ac.membership_id, ac.Month_Year, ac.received_amount, ac.bonus_amount,
        ac.Net_Advance, ac.Net_Arrears, md.cover_type, md.cover_state,
        ac.con_discount, d.overall_agediscount, d.corporate_discount,
        d.mem1_loading, d.mem2_loading
),

product_rates AS (
    SELECT
        cover_state, cover_type, product_id,
        ISNULL(CAST(MAX(CASE WHEN fee_from_date = '{curr_ri}' THEN product_fee_amount * 52.0 / 12 END) AS decimal(18,2)), 0) AS curr_mrate,
        ISNULL(CAST(MAX(CASE WHEN fee_from_date = '{prev_ri}' THEN product_fee_amount * 52.0 / 12 END) AS decimal(18,2)), 0) AS prev_mrate
    FROM dbo.product_fee
    GROUP BY cover_state, cover_type, product_id
),

Latest_payment_date AS (
    SELECT membership_id, MAX(old_paid_to) AS old_paidto
    FROM dbo.receipt
    WHERE create_datetime >= EOMONTH('{cm}', -24)
    GROUP BY membership_id
),

All_Data AS (
    SELECT
        a.*, c.old_paidto,
        CASE WHEN c.old_paidto < '{curr_ri}' THEN 1 ELSE 0 END AS RP_flag,
        ISNULL(b1.curr_mrate, 0) AS curr_H_rate, ISNULL(b1.prev_mrate, 0) AS prev_H_rate,
        ISNULL(b2.curr_mrate, 0) AS curr_A_rate, ISNULL(b2.prev_mrate, 0) AS prev_A_rate,
        ISNULL(b3.curr_mrate, 0) AS curr_B_rate, ISNULL(b3.prev_mrate, 0) AS prev_B_rate
    FROM Combined_data a
    LEFT OUTER JOIN Latest_payment_date c ON a.membership_id = c.membership_id
    LEFT OUTER JOIN product_rates b1 ON a.cover_state = b1.cover_state AND a.cover_type = b1.cover_type AND b1.product_id = a.H_prod
    LEFT OUTER JOIN product_rates b2 ON a.cover_state = b2.cover_state AND a.cover_type = b2.cover_type AND b2.product_id = a.A_prod
    LEFT OUTER JOIN product_rates b3 ON a.cover_state = b3.cover_state AND a.cover_type = b3.cover_type AND b3.product_id = a.B_prod
)

SELECT
    Month_Year, membership_id, cover_state, cover_type,
    H_prod, A_prod, B_prod,
    earned_contributions, bonus_amount,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_H_rate * (1 + overall_lhc - con_discount) * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
         ELSE CAST(((prev_H_rate * (1 + overall_lhc - con_discount) * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
    END AS H_earned,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_A_rate * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
         ELSE CAST(((prev_A_rate * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
    END AS A_earned,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_B_rate * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
         ELSE CAST(((prev_B_rate * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * earned_contributions AS decimal(18,2))
    END AS B_earned,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_H_rate * (1 + overall_lhc - con_discount) * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
         ELSE CAST(((prev_H_rate * (1 + overall_lhc - con_discount) * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
    END AS H_bonus,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_A_rate * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
         ELSE CAST(((prev_A_rate * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
    END AS A_bonus,
    CASE WHEN RP_flag = 0
         THEN CAST(((curr_B_rate * (1 - corporate_discount - overall_agediscount)) / ((curr_H_rate * (1 + overall_lhc - con_discount) + curr_A_rate + curr_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
         ELSE CAST(((prev_B_rate * (1 - corporate_discount - overall_agediscount)) / ((prev_H_rate * (1 + overall_lhc - con_discount) + prev_A_rate + prev_B_rate) * (1 - corporate_discount - overall_agediscount) + 0.0001)) * bonus_amount AS decimal(18,2))
    END AS B_bonus
FROM All_Data
"""

        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        print(f"  current_month={cm}  →  report_month={rm}  |  rows={len(df):,}")
        dfs.append(df)
        current_month += relativedelta(months=1)

    result = pd.concat(dfs, ignore_index=True)
    print(f"\nDone. Total rows: {len(result):,}  across {len(dfs)} month(s).")
    return result




In [22]:
# ---------- 使用 ----------
period_start = "2025-05-01"
period_end   = "2026-01-01"

df_rolling = run_earned_contributions(period_start, period_end, conn)
df_rolling

  current_month=2025-05-01  →  report_month=2025-04-01  |  rows=68,541
  current_month=2025-06-01  →  report_month=2025-05-01  |  rows=68,477
  current_month=2025-07-01  →  report_month=2025-06-01  |  rows=68,545
  current_month=2025-08-01  →  report_month=2025-07-01  |  rows=68,603
  current_month=2025-09-01  →  report_month=2025-08-01  |  rows=68,533
  current_month=2025-10-01  →  report_month=2025-09-01  |  rows=68,644
  current_month=2025-11-01  →  report_month=2025-10-01  |  rows=68,769
  current_month=2025-12-01  →  report_month=2025-11-01  |  rows=68,908
  current_month=2026-01-01  →  report_month=2025-12-01  |  rows=68,665

Done. Total rows: 617,685  across 9 month(s).


,Month_Year,membership_id,cover_state,cover_type,H_prod,A_prod,B_prod,earned_contributions,bonus_amount,H_earned,A_earned,B_earned,H_bonus,A_bonus,B_bonus
0,2025-04-01,229930.0,SA,F,0.0,186.0,0.0,109.24,0.0,0.00,109.24,0.0,0.0,0.0,0.0
1,2025-04-01,213.0,NSW,S,0.0,29.0,0.0,356.20,0.0,0.00,356.20,0.0,0.0,0.0,0.0
2,2025-04-01,20818.0,NSW,S,30.0,0.0,0.0,352.29,0.0,352.29,0.00,0.0,0.0,0.0,0.0
3,2025-04-01,74146.0,NSW,F,30.0,0.0,0.0,726.61,0.0,726.61,0.00,0.0,0.0,0.0,0.0
4,2025-04-01,67696.0,QLD,C,30.0,0.0,0.0,722.80,0.0,722.80,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617680,2025-12-01,181223.0,ACT,S,152.0,0.0,0.0,171.87,0.0,171.87,0.00,0.0,0.0,0.0,0.0
617681,2025-12-01,231528.0,NSW,S,0.0,166.0,0.0,301.59,0.0,0.00,301.59,0.0,0.0,0.0,0.0
617682,2025-12-01,224542.0,QLD,S,0.0,33.0,0.0,289.68,0.0,0.00,289.68,0.0,0.0,0.0,0.0
617683,2025-12-01,225099.0,NSW,C,0.0,156.0,0.0,145.90,0.0,0.00,145.90,0.0,0.0,0.0,0.0


In [23]:
df_rolling.to_csv(os.path.join(output_folder, "historical_ec.csv"), index=False)